In [103]:
pip install --upgrade tensorflow numpy

Note: you may need to restart the kernel to use updated packages.Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.25.2-cp310-cp310-win_amd64.whl (15.6 MB)



In [104]:
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.python.keras.utils.np_utils import to_categorical

In [94]:
DATA_PATH = os.path.join('MP_Data') 

actions = np.array(['homem','obrigado', 'oi'])

max_frames_per_video = 200
numeros_videos_por_action = 3
label_map = {label:num for num, label in enumerate(actions)}

In [95]:
sequences, labels = [], []

for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(max_frames_per_video):
            file_path = os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num))
            
            try:
                res = np.load(file_path)
            except FileNotFoundError:
                array_shape = (258,)
                zeros_array = np.zeros(array_shape)
                window.append(zeros_array)
            else:
                window.append(res)
        
        sequences.append(window)
        labels.append(label_map[action])

        


In [96]:

y = np.array(labels)
print(y.shape)
X = np.array(sequences)
print(X.shape)
y = to_categorical(labels).astype(int)


(9,)
(9, 200, 258)


In [97]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)


In [98]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [99]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
actions.shape

(3,)

In [100]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(200,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [101]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [102]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
1/1 [==============================] - 12s 12s/step - loss: 2124.7576 - categorical_accuracy: 0.1667
Epoch 2/2000
1/1 [==============================] - 1s 618ms/step - loss: 414.5475 - categorical_accuracy: 0.0000e+00
Epoch 3/2000
1/1 [==============================] - 1s 520ms/step - loss: nan - categorical_accuracy: 0.3333
Epoch 4/2000
1/1 [==============================] - 1s 539ms/step - loss: nan - categorical_accuracy: 0.3333
Epoch 5/2000
1/1 [==============================] - 1s 542ms/step - loss: nan - categorical_accuracy: 0.3333
Epoch 6/2000
1/1 [==============================] - 1s 1s/step - loss: nan - categorical_accuracy: 0.3333
Epoch 7/2000
1/1 [==============================] - 1s 523ms/step - loss: nan - categorical_accuracy: 0.3333
Epoch 8/2000
1/1 [==============================] - 1s 557ms/step - loss: nan - categorical_accuracy: 0.3333
Epoch 9/2000
1/1 [==============================] - 1s 519ms/step - loss: nan - categorical_accuracy: 0.3333
Epoch 10

KeyboardInterrupt: 